# Delphi Demo - CAGs from <span style='color:royalblue; font-style: italic'>Text</span>

*July 30, 2018*

This is a Jupyter notebook created to showcase the design and capabilities of
the Delphi package, available at [https://github.com/ml4ai/delphi](https://github.com/ml4ai/delphi). 

A rendered HTML version of this notebook can also be found at
[`http://vision.cs.arizona.edu/adarsh/Delphi-Demo-Notebook.html`](http://vision.cs.arizona.edu/adarsh/export/Delphi-Demo-Notebook.html)

In [ ]:
def pythonshell():
    """Determine python shell

    pythonshell() returns

    'shell' (started python on command line using "python")
    'ipython' (started ipython on command line using "ipython")
    'ipython-notebook' (e.g., running in Spyder or started with "ipython qtconsole")
    'jupyter-notebook' (running in a Jupyter notebook)

    See also https://stackoverflow.com/a/37661854
    """

    import os
    env = os.environ
    shell = 'shell'
    program = os.path.basename(env['_'])

    if 'jupyter-notebook' in program:
        shell = 'jupyter-notebook'
    elif 'JPY_PARENT_PID' in env or 'ipython' in program:
        shell = 'ipython'
        if 'JPY_PARENT_PID' in env:
            shell = 'ipython-notebook'

    return shell

This demo has been tested with the version of Delphi corresponding to the commit hash below.

In [ ]:
import subprocess as sp
if pythonshell() == "ipython-notebook":
    commit_hash = sp.check_output(["git", "rev-parse", "HEAD"])
    print(commit_hash[:-1])

## Construct and visualize CAG corresponding to use case

In [ ]:

from delphi.utils import get_data_from_url
import urllib.request as request
import pickle
from delphi.AnalysisGraph import AnalysisGraph
from delphi.visualization import visualize
from delphi.assembly import get_valid_statements_for_modeling
import pandas as pd
from delphi.inspection import statements
import delphi.jupyter_tools as jt
if pythonshell() == "ipython-notebook":
    %matplotlib inline
    %load_ext autoreload
    %autoreload 2

In [ ]:
from delphi.utils.indra import get_statements_from_json_dict
import json
url = "http://vision.cs.arizona.edu/adarsh/export/demos/data/preassembled_indra_statements-small.json"
sts = get_statements_from_json_dict(json.loads(request.urlopen(url).read()))

In [ ]:
G = AnalysisGraph.from_statements(sts)

In [ ]:
G.merge_nodes('UN/entities/human/food/food_security', 'UN/entities/human/food/food_insecurity', same_polarity=False)

In [ ]:
concepts = ["UN/events/weather/precipitation", "UN/entities/human/food/food_insecurity", "UN/events/human/conflict"]
G = G.get_subgraph_for_concept_pairs(concepts, cutoff=2)

In [ ]:
visualize(G, rankdir='TB', nodes_to_highlight=concepts)

## Inspecting and editing CAGs

## Inspecting statements

In [ ]:
pd.options.display.max_colwidth=1000
pd.options.display.width=1000
jt.create_statement_inspection_table(statements(G))

## Removing incorrect edges

In [ ]:
G.remove_edges_from([('UN/entities/human/food/food_insecurity', 'UN/events/natural_disaster/drought'), ('UN/entities/human/food/food_insecurity', 'UN/entities/human/food/food_insecurity')])
visualize(G, rankdir='TB', nodes_to_highlight='UN/entities/human/food/food_insecurity')

## Mapping concepts to indicators

In [ ]:
url = 'http://vision.cs.arizona.edu/adarsh/export/demos/data/concept_to_indicator_mapping.txt'
mapping = get_data_from_url(url)
G.map_concepts_to_indicators(1, mapping)
visualize(G, indicators=True)

In [ ]:
from datetime import datetime
url = 'http://vision.cs.arizona.edu/adarsh/12_month_evaluation_indicator_data.csv'
df = pd.read_csv(url)

In [ ]:
date = datetime(2014,1,1)
G.parameterize(date, df)

In [ ]:
visualize(G, indicators=True, indicator_values=True,
          graph_label=f'Causal Analysis Graph for South Sudan, {date.year}')

## Infer transition model

In [ ]:
adjective_data_url = 'http://vision.cs.arizona.edu/adarsh/export/demos/data/adjectiveData.tsv'
G.assemble_transition_model_from_gradable_adjectives(request.urlopen(adjective_data_url), 100)

## Set initial parameters

In [ ]:
G.create_bmi_config_file()
s0 = pd.read_csv('bmi_config.txt', index_col=0, header=None,
                 error_bad_lines=False)[1]
s0.loc['∂(UN/events/human/conflict)/∂t'] = 0.1
s0.to_csv('bmi_config.txt')
s0

## Execute model

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from delphi.utils.misc import _insert_line_breaks

G.initialize()
concept1 = 'UN/events/human/conflict'
concept2 = 'UN/entities/human/food/food_insecurity'

ind1 = list(G.nodes[concept1]['indicators'].values())[0]
ind2 = list(G.nodes[concept2]['indicators'].values())[0]

plot_data = {variable: {'xs':[], 'ys':[],
                        'ylabel': variable.replace('_', ' ').capitalize(), 'units': '',
                       'ax_number': i} 
             for i, variable in enumerate((concept1, concept2, ind1.name, ind2.name))}

plot_data[ind1.name]['units'] = f"({ind1.unit})"
plot_data[ind2.name]['units'] = f"({ind2.unit})"

n_timesteps = 5
for day in range(n_timesteps):
    G.update()
    for concept in (concept1, concept2):
        for datapoint in G.nodes[concept]['rv'].dataset:
            plot_data[concept]['xs'].append(day+1)
            plot_data[concept]['ys'].append(datapoint)
        indicator = list(G.nodes[concept]['indicators'].values())[0]
        for datapoint in indicator.dataset:
            if indicator.name != ind2.name:
                plot_data[indicator.name]['xs'].append(day+1)
                plot_data[indicator.name]['ys'].append(datapoint)
            else:
                if 0 < datapoint < 100:
                    plot_data[indicator.name]['xs'].append(day+1)
                    plot_data[indicator.name]['ys'].append(datapoint)
                    
        


fig, axes = plt.subplots(1,4, figsize=(20, 4))
for variable, plot_data in plot_data.items():
    ax = axes[plot_data['ax_number']]
    ax.set_xlabel('Time step')
    ax.set_xticks(range(6))
    ax.set_title(_insert_line_breaks(' '.join((plot_data['ylabel'], plot_data['units']))))
    sns.lineplot(plot_data['xs'], plot_data['ys'], ax = ax)

Still todo: truncate probability distributions from 0 to 100 for percentages.

In [ ]:
from delphi.inspection import inspect_edge
inspect_edge(G, 'UN/events/human/conflict', 'UN/entities/human/food/food_insecurity')

## Exploring the unknown unknowns

In [ ]:
url = 'http://vision.cs.arizona.edu/adarsh/export/demos/data/pi_mtg_demo_unfiltered.pkl'
sts=pickle.load(get_data_from_url(url))

In [ ]:
G = AnalysisGraph.from_statements(sts)
G.merge_nodes('UN/entities/human/food/food_security', 'UN/entities/human/food/food_insecurity', same_polarity=False)
G = G.get_subgraph_for_concept_pair('UN/events/natural_disaster/drought', 'UN/entities/human/food/food_insecurity', cutoff=3)
visualize(G, nodes_to_highlight=['UN/events/natural_disaster/drought','UN/entities/human/food/food_insecurity'])

# Causal analysis graphs from <span style='color:royalblue; font-style: italic'>Software</span>

## Original Fortran program

In [ ]:
if pythonshell() == "ipython-notebook":
    jt.display('../tests/data/crop_yield.f')

## Executable DBN - Loop plate representation

In [ ]:
from delphi.GrFN.scopes import Scope
if pythonshell() == "ipython-notebook":
    A = Scope.from_fortran_file("../tests/data/crop_yield.f").to_agraph()
    jt.display_image(A.draw(format='png', prog='dot'))
else:
    A = Scope.from_fortran_file("tests/data/crop_yield.f").to_agraph()

## High-level representation of CAG from program

In [ ]:
from importlib import import_module
from delphi.GrFN.ProgramAnalysisGraph import ProgramAnalysisGraph
G = ProgramAnalysisGraph.from_agraph(A, import_module("crop_yield_lambdas"))
G.initialize()
from delphi.visualization import visualize
visualize(G, show_values = True)

In [ ]:
G.update()
visualize(G, show_values = True)

In [ ]:
G.update()
visualize(G, show_values = True)

## Sensitivity Analysis

In [ ]:
import seaborn as sns
sns.set_style('darkgrid')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import numpy as np
from matplotlib import pyplot as plt
from delphi.utils import compose, rcompose
from delphi.GrFN.ProgramAnalysisGraph import ProgramAnalysisGraph

def make_plots(n_samples, deterministic = True):
    variables = ('RAIN', 'TOTAL_RAIN', 'YIELD_EST')
    vals = {k:[] for k in variables}
    days = {k:[] for k in variables}
    palette = sns.color_palette()
    colors = {k:palette[i] for i, k in enumerate(vals)}
    fig, axes = plt.subplots(1,len(vals), figsize=(18, 5))
    ax = {k:axes[i] for i, k in enumerate(vals)}

    for _ in range(n_samples):
        G = ProgramAnalysisGraph.from_agraph(A, import_module("crop_yield_lambdas"))
        if not deterministic:
            G.nodes['MAX_RAIN']['init_fn'] = lambda: np.random.normal(4, 1)
        G.initialize()
        for i in range(1,31):
            G.update()
            for k in vals:
                vals[k].append(G.nodes[k]['value'])
                days[k].append(G.nodes['DAY']['value'])

    for k in vals:
        sns.lineplot(days[k], vals[k], ax = ax[k], label=k, color=colors[k])
        ax[k].set_xlabel('DAY', fontsize=20)
        ax[k].set_ylabel(k, fontsize=20)

    plt.tight_layout()

make_plots(10, deterministic=False)